In [1]:
#Imports
from comet_ml import Experiment
experiment = Experiment(api_key="YkPEmantOag1R1VOJmXz11hmt", parse_args=False, project_name='SegNet')

COMET INFO: old comet version (1.0.44) detected. current: 1.0.51 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET WARNING: Failing to collect the installed os packages
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/yikeqicn/segnet/123aad549e0c4a6b8e47b4f1bcf75c1c



In [2]:
from os.path import join, basename, dirname
import tensorflow as tf
import shutil
import os
import sys
import argparse

import cv2
import numpy as np
import os
import time

from torch.utils.data import DataLoader, ConcatDataset, random_split#, SequentialSampler #yike: add SequentialSampler
import torch
import torchvision
import torchvision.transforms as transforms
from datasets import ArtPrint, ArtPrintNoIntsect
from Model import *

home = os.environ['HOME']

In [3]:
#General Settings
parser = argparse.ArgumentParser()

# system basics
parser.add_argument("-name", default='debug', type=str, help="name of the log")
parser.add_argument("-gpu", default='0', type=str, help="gpu numbers")

parser.add_argument("-train", default=True, help="train the NN", action="store_true")

parser.add_argument("-transfer",default=False, help="test the NN", action="store_true")

parser.add_argument("-test",default=False, help="test the NN", action="store_true")

# image and logistic parameters 
parser.add_argument("-image_h", default=32, type=int, help='image height') #('image_h', "360", """ image height """) 32
parser.add_argument("-image_w", default=128, type=int, help='image width')#('image_w', "480", """ image width """)128
#parser.add_argument("-image_h", default=360, type=int, help='image height') 
#parser.add_argument("-image_w", default=480, type=int, help='image width')

parser.add_argument("-image_c", default=1, type=int, help='image channel')#('image_c', "3", """ image channel (RGB) """)
parser.add_argument("-num_class", default=2, type=int, help='total class number')

# training hyperparam
parser.add_argument("-batch_size", default=10, type=int, help='batch_size')
parser.add_argument("-lrInit", default=1e-3, type=int, help='initial lr')
parser.add_argument("-lrDrop1", default=10, type=int, help='step to drop lr by 10 first time') # not sure
parser.add_argument("-lrDrop2", default=1000, type=int, help='step to drop lr by 10 sexond time') # not sure
parser.add_argument('-max_epoch',default=3, type=int,help='max epoch numbers')



# file paths
parser.add_argument('-ckpt_root', default="/home/classify_copy/misc_test/ckpt", type=str,help= "dir to store ckpt") # log_dir !!!!!
parser.add_argument('-data_root', default="/home/classify_copy/misc_test/datasets", type=str, help=" root to any data folder ")
parser.add_argument('-urlTranferFrom', default="", type=str, help=" archived model url ")


#args = parser.parse_args()
args = parser.parse_known_args()[0]

name = args.name

experiment.set_name(name)
experiment.log_parameters(vars(args))

os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

ckptroot = args.ckpt_root
args.ckptpath = join(ckptroot, name)
if args.name=='debug': shutil.rmtree(args.ckptpath, ignore_errors=True)
os.makedirs(args.ckptpath, exist_ok=True)


In [4]:
def checkArgs():
    if args.test:
        print('The model is set to Testing')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid testing dir: %s"%FLAGS.test_dir)
    elif args.transfer:
        print('The model is set to transfer learn from ckpt')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Image dir: %s"%FLAGS.image_dir)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)
    else:
        print('The model is set to Training')
        print("Max training Iteration: %d"%args.max_epoch)
        print("Initial lr: %f"%args.lrInit)
        print("First Drop Steps: %i"%args.lrDrop1)
        print("Second Drop Steps: %i"%args.lrDrop2)
        print("Data root: %s"%args.data_root)
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)

    print("Batch Size: %d"%args.batch_size)

In [5]:
#main function
def main():    
    checkArgs()
    if args.train:
        transform_train = transforms.Compose([
          transforms.Lambda(lambda img: cv2.resize(img, (args.image_w,args.image_h), interpolation=cv2.INTER_CUBIC)),
          transforms.Lambda(lambda img: np.expand_dims(img,3) ),
          #transforms.Lambda(lambda img: add_artifacts(img,args)),
          #transforms.Lambda(lambda img: cv2.transpose(img))
        ])
        #arprint=ArtPrint(args.data_root, transform=transform_train)
        arprint=ArtPrintNoIntsect(args.data_root, transform=transform_train)
        concat=arprint
        idxTrain = int( len(arprint)*0.9)
        trainset, testset = random_split(concat, [idxTrain, len(concat)-idxTrain])
        trainloader = DataLoader(trainset, batch_size=args.batch_size, shuffle=True, drop_last=True,num_workers=4)
        testloader = DataLoader(testset, batch_size=args.batch_size, shuffle=False, drop_last=False,num_workers=2)
        
        #weight gen
        pos_perc=sum(map(lambda x: np.sum(cv2.imread(x[1],cv2.IMREAD_GRAYSCALE)),trainset.dataset.samples))/(args.image_h*args.image_w*len(trainset.dataset.samples))
        neg_perc=1-pos_perc
        weight=np.array([pos_perc,neg_perc]) # just reverse
        print(weight)
        model=Model(args, experiment, loss_weight=weight, mustRestore=False)
        model.training(loader=trainloader, validateloader=testloader)
        
    else:
        pass # for now, leave it pass 
    

In [6]:
main()

The model is set to Training
Max training Iteration: 3
Initial lr: 0.001000
First Drop Steps: 10
Second Drop Steps: 1000
Data root: /home/classify_copy/misc_test/datasets
Check point file: /home/classify_copy/misc_test/ckpt/debug
Batch Size: 10
/home/classify_copy/misc_test/datasets/artifact_images_no_intersect already exists, skipping download
[0.05999042 0.94000958]
Instructions for updating:
Colocations handled automatically by placer.
GGG
(?, 32, 128, 1)
(?, 32, 128, 1)
(?, 32, 128, 64)
111111
(?, 16, 64, 64)
(?, 16, 64, 64)
22222
(?, 8, 32, 64)
(?, 8, 32, 64)
33333
(?, 4, 16, 64)
(?, 4, 16, 64)
44444
(?, 2, 8, 64)
(?, 2, 8, 64)
d4444444
(?, 4, 16, 64)
d333333
(?, 8, 32, 64)
d22222
(?, 16, 64, 64)
d111111
(?, 32, 128, 64)
cv
(?, 32, 128, 2)
(2,)
(?, 32, 128, 2)
loss: ()
INFO:tensorflow:Summary name conv_classifier/weight_loss (raw) is illegal; using conv_classifier/weight_loss__raw_ instead.
INFO:tensorflow:Summary name loss/cross_entropy (raw) is illegal; using loss/cross_entropy_

COMET ERROR: Failed to extract parameters from Estimator.init()


toto_loss_shape: Tensor("loss/total_loss:0", shape=(), dtype=float32)
Instructions for updating:
Use tf.cast instead.
Python: 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
Tensorflow: 1.12.0-rc0
Ran global_variables_initializer
Epoch: 1  Training...


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys


TRAIN: Batch: 0.0 Loss: 0.10882363
accuracy = 0.415710
mean IU  = 0.234034
    class # 0 accuracy = 0.384415 
    class # 1 accuracy = 0.882925 
TRAIN: Batch: 0.003908998514580564 Loss: 0.070077255
accuracy = 0.603800
mean IU  = 0.351608
    class # 0 accuracy = 0.587987 
    class # 1 accuracy = 0.822932 
TRAIN: Batch: 0.007817997029161129 Loss: 0.062187124
accuracy = 0.708953
mean IU  = 0.421869
    class # 0 accuracy = 0.700252 
    class # 1 accuracy = 0.842231 
TRAIN: Batch: 0.011726995543741693 Loss: 0.054128405
accuracy = 0.709539
mean IU  = 0.410370
    class # 0 accuracy = 0.707177 
    class # 1 accuracy = 0.750447 
TRAIN: Batch: 0.015635994058322257 Loss: 0.054726854
accuracy = 0.707532
mean IU  = 0.428408
    class # 0 accuracy = 0.697121 
    class # 1 accuracy = 0.846856 
TRAIN: Batch: 0.019544992572902823 Loss: 0.05978281
accuracy = 0.617045
mean IU  = 0.360643
    class # 0 accuracy = 0.601748 
    class # 1 accuracy = 0.830113 
TRAIN: Batch: 0.023453991087483386 Loss: 

...

TRAIN: Batch: 0.5980767727308264 Loss: 0.01585494
accuracy = 0.945741
mean IU  = 0.729780
    class # 0 accuracy = 0.943690 
    class # 1 accuracy = 0.978216 
TRAIN: Batch: 0.601985771245407 Loss: 0.023696266
accuracy = 0.911799
mean IU  = 0.668939
    class # 0 accuracy = 0.907131 
    class # 1 accuracy = 0.974849 
TRAIN: Batch: 0.6058947697599875 Loss: 0.023741674
accuracy = 0.921666
mean IU  = 0.648434
    class # 0 accuracy = 0.921821 
    class # 1 accuracy = 0.918856 
TRAIN: Batch: 0.6098037682745681 Loss: 0.021895418
accuracy = 0.933324
mean IU  = 0.661112
    class # 0 accuracy = 0.932566 
    class # 1 accuracy = 0.949299 
TRAIN: Batch: 0.6137127667891487 Loss: 0.022620596
accuracy = 0.916744
mean IU  = 0.628523
    class # 0 accuracy = 0.915489 
    class # 1 accuracy = 0.942571 
TRAIN: Batch: 0.6176217653037291 Loss: 0.017374076
accuracy = 0.945865
mean IU  = 0.736417
    class # 0 accuracy = 0.944130 
    class # 1 accuracy = 0.971683 
TRAIN: Batch: 0.6215307638183097 Los

TRAIN: Batch: 0.7974356969744352 Loss: 0.025932763
accuracy = 0.927340
mean IU  = 0.698266
    class # 0 accuracy = 0.926310 
    class # 1 accuracy = 0.941094 
TRAIN: Batch: 0.8013446954890158 Loss: 0.021677952
accuracy = 0.927921
mean IU  = 0.691957
    class # 0 accuracy = 0.926203 
    class # 1 accuracy = 0.952777 
TRAIN: Batch: 0.8052536940035963 Loss: 0.024708208
accuracy = 0.934365
mean IU  = 0.704262
    class # 0 accuracy = 0.935534 
    class # 1 accuracy = 0.917722 
TRAIN: Batch: 0.8091626925181769 Loss: 0.024507426
accuracy = 0.935681
mean IU  = 0.691500
    class # 0 accuracy = 0.937144 
    class # 1 accuracy = 0.911889 
TRAIN: Batch: 0.8130716910327574 Loss: 0.016611356
accuracy = 0.947695
mean IU  = 0.745042
    class # 0 accuracy = 0.946825 
    class # 1 accuracy = 0.960104 
TRAIN: Batch: 0.8169806895473379 Loss: 0.017758802
accuracy = 0.945212
mean IU  = 0.727595
    class # 0 accuracy = 0.944151 
    class # 1 accuracy = 0.961804 
TRAIN: Batch: 0.8208896880619185 L

TRAIN: Batch: 0.996794621218044 Loss: 0.023863604
accuracy = 0.929351
mean IU  = 0.663297
    class # 0 accuracy = 0.929112 
    class # 1 accuracy = 0.933848 
Validating NN


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys


VALID: Total accuracy: 93.112530%. Class 0 capture: 92.918399%. Class 1 capture: 96.178018%
Character error rate improved, save model
Done with training at epoch 3 sigoptObservation=0.9311252995621195
